In [ ]:
# Importing numpy, pandas, math and matplotlib libraries and setting default fonts

import numpy as np
import pandas as pd
import os
import math
import decimal
import matplotlib
import matplotlib.image as image
from matplotlib import artist
from matplotlib import pyplot as plt
from matplotlib import rcParams
import ipywidgets
from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual,HBox,fixed
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
layout = widgets.Layout(width='400px', height='30px')
pd.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Franklin Gothic Medium', 'Franklin Gothic Book']
%matplotlib inline

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
Rotelle = pd.read_csv('Player_Stats_PL.csv', index_col=0) #husk å sette denne til 1 om det ikke funker

Rotelle = Rotelle.fillna(0)

# To display all columns and rows
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 

Rotelle['90s'] = Rotelle['minutes']/90

In [ ]:
minutes_position_filtered = (Rotelle['minutes'] >= 90)  & (Rotelle['position'].str.startswith('FW'))
df = Rotelle.loc[minutes_position_filtered, ['player', 'squad', 'minutes', 'position', 'passes_pct', 'progressive_passes', 'passes_into_final_third', 'passes_into_penalty_area', 'assisted_shots','passes_pressure', 'miscontrols', 'dispossessed', 'dribbles_completed_pct', 'dribbles_completed', 'carry_progressive_distance', 'carries', 'carries_into_final_third','touches_att_3rd', 'touches_att_pen_area','carries_into_penalty_area', 'pass_targets', 'passes_received_pct', 'passes_received', 'gca_per90', 'sca_per90', 'xg_per90', 'npxg_per90','xa_per90', 'npxg_xa_per90', 'xa_net', 'npxg_net','npxg_per_shot', 'shots_on_target_pct', 'aerials_won_pct','aerials_won', 'tackles_won', 'tackles', 'interceptions', 'ball_recoveries', 'pressure_regain_pct', 'dribbled_past', '90s']] #Do not touch this line!!
minutes_filteret = df.set_index('player')

minutes_filteret.head(5)

In [ ]:
passing_percentiles = pd.DataFrame()
passing_percentiles['squad'] = minutes_filteret['squad']

#Passing stats
passing_percentiles['Pass\nCompletion %'] = minutes_filteret['passes_pct'].rank(pct=True)*100
passing_percentiles['Pressured Passes\nPer 90'] = minutes_filteret['passes_pressure'].rank(pct=True)*100
passing_percentiles['Progressive Passes\nPer 90'] = \
    (minutes_filteret['progressive_passes']/minutes_filteret['90s']).rank(pct=True)*100
passing_percentiles['Successful Deliveries\nInto Box Per 90'] = \
    (minutes_filteret['passes_into_penalty_area']/minutes_filteret['90s']).rank(pct=True)*100
passing_percentiles['Key Passes\n Per 90'] = \
    (minutes_filteret['assisted_shots']/minutes_filteret['90s']).rank(pct=True)*100


#Possesion stats
passing_percentiles['Rate Adj Target of an\nAttempted Pass'] =\
    (minutes_filteret['passes_received']*minutes_filteret['passes_received_pct']).rank(pct=True)*100     
passing_percentiles['Turnovers\nPer 90'] = \
    ((minutes_filteret['miscontrols']+minutes_filteret['dispossessed'])/minutes_filteret['90s']).rank(pct=True, ascending=False)*100
passing_percentiles['Rate Adj Successful\nDribbles Per 90'] =\
    (minutes_filteret['dribbles_completed']*minutes_filteret['dribbles_completed_pct']/ \
    minutes_filteret['90s']).rank(pct=True)*100                                          
passing_percentiles['Progressive Distance\nPer Carry'] = \
    (minutes_filteret['carry_progressive_distance']/minutes_filteret['carries']).rank(pct=True)*100
passing_percentiles['Penalty Area\nEntries'] = minutes_filteret['carries_into_penalty_area'].rank(pct=True)*100
passing_percentiles['Touches in\nBox'] = minutes_filteret['touches_att_pen_area'].rank(pct=True)*100


#Contribution stats
passing_percentiles['GCA\nPer 90'] = minutes_filteret['gca_per90'].rank(pct=True)*100
passing_percentiles['SCA\nPer 90'] = minutes_filteret['sca_per90'].rank(pct=True)*100
passing_percentiles['xA\nPer 90'] = minutes_filteret['xa_per90'].rank(pct=True)*100
passing_percentiles['Non-Penalty\nxG Per 90'] = minutes_filteret['npxg_per90'].rank(pct=True)*100
passing_percentiles['Non-Penalty\nxA + xG Per 90'] = minutes_filteret['npxg_xa_per90'].rank(pct=True)*100 
passing_percentiles['xG\nPer Shot'] = minutes_filteret['npxg_per_shot'].rank(pct=True)*100
passing_percentiles['Netto xG'] = minutes_filteret['npxg_net'].rank(pct=True)*100 


#Defening stats
passing_percentiles['Rate Adj\nAerials Won %'] = \
    (minutes_filteret['aerials_won']*minutes_filteret['aerials_won_pct']).rank(pct=True)*100     
passing_percentiles['Ball recoveries\nPer 90'] = \
    (minutes_filteret['ball_recoveries']/minutes_filteret['90s']).rank(pct=True)*100    
passing_percentiles['Interceptions\nPer 90'] = \
    (minutes_filteret['interceptions']/minutes_filteret['90s']).rank(pct=True)*100 
passing_percentiles['Pressure\nRegain %'] = minutes_filteret['pressure_regain_pct'].rank(pct=True)*100
passing_percentiles['Rate Adj Tackles\n Won %'] = \
    ((minutes_filteret['tackles_won'])*(minutes_filteret['tackles_won']/minutes_filteret['tackles'])).rank(pct=True)*100
passing_percentiles['Dribbled Past'] = minutes_filteret['dribbled_past'].rank(pct=True, ascending=False)*100
                                          
passing_percentiles['minutes'] = minutes_filteret['minutes']
passing_percentiles['position'] = minutes_filteret['position']
passing_percentiles['90s'] = minutes_filteret['90s']

In [ ]:
passing_percentiles['minutes'] = passing_percentiles['minutes'].astype(int)
passing_percentiles['minutes'] = passing_percentiles['minutes'].astype(str)
passing_percentiles = passing_percentiles.fillna(0)
passing_percentiles.head(500)

In [ ]:
def label_adjuster(i):
    
    if i <= 50: #Everything under this percentile will be pushed to a 100 - outside the chart
        return 101
    else:
        return -10

In [ ]:
def percentile_chart(player_name):
    
    # Creates bars based on the player entered
    theta = list(map((lambda x: x*math.pi/24), list(range(1,49,2))))
    radii = list(passing_percentiles.loc[player_name][1:25])
    width = [math.pi/12]*len(radii)
    
    # Aesthetic details of the figure including color, axis tick size, x value specifics
    # and r-axis limits, etc.
    plt.figure(figsize=(10,10), facecolor = '#37474f')
    ax = plt.subplot(111, projection='polar')
    ax.set_facecolor('#37474f')
    #ax = plt.subplot(111, projection='polar')
    ax.grid(True)
    ax.xaxis.grid(linewidth=2.5, color='xkcd:white')
    ax.yaxis.grid(linewidth=1.5, color='xkcd:white', alpha=0.3)
    ax.tick_params(axis='x', colors='#37474f')
    ax.tick_params(axis='y', colors='#37474f')
    ax.set_rlim(0,100)
    ax.spines['polar'].set_color('xkcd:white')
    ax.spines['polar'].set_linewidth(3)
    ax.set_rlabel_position(0)
    ax.set_rticks([25,50,75])
    ax.tick_params(labelsize=0)
    labels = plt.thetagrids(range(0, 360, 15), ('','','','','','','','','','','','','','','','','','','','','','','',''))
    ax.spines['polar'].set_color('xkcd:white')
    ax.spines['polar'].set_linewidth(2.5)
    ax.set_rorigin(-15)
  
   
    # Aesthetic details of the bars and chart titles based on player name
    ax.bar(theta, radii, width, color=['#ffba08','#ffba08','#ffba08','#ffba08','#ffba08','#219ebc','#219ebc','#219ebc','#219ebc','#219ebc','#219ebc','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#2a9d8f','#2a9d8f', '#2a9d8f', '#2a9d8f', '#2a9d8f','#2a9d8f'], \
           alpha=0.7, edgecolor='xkcd:white', linewidth=0.5)
    ax.set_title('\n'+player_name+' - Performance Overview\n\n\n', \
                 color='xkcd:white', fontsize=26) 
    plt.text(0.5, 1.13, "Position: " + r"$\bf{" + passing_percentiles.at[player_name, 'position'] + "}$" + " || " + " Minutes Played: " + r"$\bf{" + passing_percentiles.at[player_name, 'minutes'] + "}$" + " || " + " Club: " + r"$\bf{" + passing_percentiles.at[player_name, 'squad'] + "}$", \
         horizontalalignment='center', verticalalignment='top', color='#ef233c', size='17', \
             transform=ax.transAxes)

    l = list(passing_percentiles)
    
    #Adds text to the chart in necessary places
    #plt.text(-0.2, -0.1, 'Data obtained from FBRef.com', \
         #horizontalalignment='left', verticalalignment='top', color='xkcd:black', \
             #transform=ax.transAxes, size='14')
    #plt.text(0.5, 1.22, "Percentile Rankings vs MF and FW in Same League, 2020/21", \
         #horizontalalignment='center', verticalalignment='top', color='#212f45', size='14', \
             #transform=ax.transAxes)
    #plt.text(1.2, -0.075, '\nExcludes players with <1200 minutes', \
        #horizontalalignment='right', verticalalignment='top', color='xkcd:black', \
             #style='italic', transform=ax.transAxes, size='14')

    
    #fixing bars lables
    plt.text(1.1, 0.57, l[1], \
         horizontalalignment='center', verticalalignment='top', color='xkcd:white',\
             transform=ax.transAxes, size='13',)
    plt.text(1.08, 0.7, l[2], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13',)
    plt.text(1, 0.83, l[3], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.91, 0.93, l[4], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.76, 1.00, l[5], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.59, 1.055, l[6], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.40, 1.055, l[7], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.24, 1.02, l[8], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.1, 0.93, l[9], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.01, 0.83, l[10], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.05, 0.7, l[11], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.09, 0.57, l[12], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.08, 0.42, l[13], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.03, 0.29, l[14], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.028, 0.16, l[15], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.13, 0.06, l[16], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.27, -0.02, l[17], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.43, -0.04, l[18], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.58, -0.04, l[19], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.73, -0.02, l[20], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.89, 0.063, l[21], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(1, 0.17, l[22], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(1.07, 0.3, l[23], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(1.09, 0.42, l[24], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')          

    
    # Annotating bar labels with their values
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]:
        plt.annotate(str(int(round(radii[i])))+"%", (theta[i], radii[i]+label_adjuster(radii[i])), color='xkcd:white', \
                horizontalalignment='center', fontsize=14)
    
   
    plt.tight_layout()
    plt.subplots_adjust(left=0.12, right=0.85, top=0.85, bottom=0.08)
    #plt.show() If saving pic this needs to be hashed out

In [ ]:
percentile_chart('Pierre-Emerick Aubameyang') 

In [ ]:
#Arsenal

percentile_chart('Pierre-Emerick Aubameyang') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Pierre-Emerick Aubameyang.png', transparent=True, dpi=72)

percentile_chart('Alexandre Lacazette') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Alexandre Lacazette.png', transparent=True, dpi=72)

percentile_chart('Martinelli') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Martinelli.png', transparent=True, dpi=72)

percentile_chart('Eddie Nketiah') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Eddie Nketiah.png', transparent=True, dpi=72)

percentile_chart('Nicolas Pépé') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Nicolas Pépé.png', transparent=True, dpi=72)

percentile_chart('Bukayo Saka') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Bukayo Saka.png', transparent=True, dpi=72)

percentile_chart('Willian') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Arsenal FC\FW\Willian.png', transparent=True, dpi=72)

In [ ]:
#Aston Villa

percentile_chart('Anwar El Ghazi') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\FW\Anwar El Ghazi.png', transparent=True, dpi=72)

percentile_chart('Jack Grealish') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\FW\Jack Grealish.png', transparent=True, dpi=72)

percentile_chart('Bertrand Traoré') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\FW\Bertrand Traoré.png', transparent=True, dpi=72)

percentile_chart('Trézéguet') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\FW\Trézéguet.png', transparent=True, dpi=72)

percentile_chart('Ollie Watkins') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\FW\Ollie Watkins.png', transparent=True, dpi=72)

In [ ]:
#Brighton

percentile_chart('Aaron Connolly') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\FW\Aaron Connolly.png', transparent=True, dpi=72)

percentile_chart('Alireza Jahanbakhsh') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\FW\Alireza Jahanbakhsh.png', transparent=True, dpi=72)

percentile_chart('Neal Maupay') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\FW\Neal Maupay.png', transparent=True, dpi=72)

percentile_chart('Leandro Trossard') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\FW\Leandro Trossard.png', transparent=True, dpi=72)

In [ ]:
#Burnley

percentile_chart('Ashley Barnes') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\FW\Ashley Barnes.png', transparent=True, dpi=72)

percentile_chart('Jay Rodriguez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\FW\Jay Rodriguez.png', transparent=True, dpi=72)

percentile_chart('Matěj Vydra') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\FW\Matěj Vydra.png', transparent=True, dpi=72)

percentile_chart('Chris Wood') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\FW\Chris Wood.png', transparent=True, dpi=72)

In [ ]:
#Chelsea

percentile_chart('Tammy Abraham') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Tammy Abraham.png', transparent=True, dpi=72)

percentile_chart('Olivier Giroud') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Olivier Giroud.png', transparent=True, dpi=72)

percentile_chart("Callum Hudson-Odoi") 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Callum Hudson-Odoi.png', transparent=True, dpi=72)

percentile_chart('Hakim Ziyech') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Hakim Ziyech.png', transparent=True, dpi=72)

percentile_chart('Christian Pulisic') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Christian Pulisic.png', transparent=True, dpi=72)

percentile_chart('Timo Werner') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\FW\Timo Werner.png', transparent=True, dpi=72)

In [ ]:
#Crystal Palace

percentile_chart('Jordan Ayew') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\FW\Jordan Ayew.png', transparent=True, dpi=72)

percentile_chart('Michy Batshuayi') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\FW\Michy Batshuayi.png', transparent=True, dpi=72)

percentile_chart('Christian Benteke') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\FW\Christian Benteke.png', transparent=True, dpi=72)

percentile_chart('Jean-Philippe Mateta') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\FW\Jean-Philippe Mateta.png', transparent=True, dpi=72)

percentile_chart('Wilfried Zaha') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\FW\Wilfried Zaha.png', transparent=True, dpi=72)

In [ ]:
#Everton

#percentile_chart('Bernard') 
#plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\FW\Bernard.png', transparent=True, dpi=72)

percentile_chart('Dominic Calvert-Lewin')
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\FW\Dominic Calvert-Lewin.png', transparent=True, dpi=72)

percentile_chart('Alex Iwobi') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\FW\Alex Iwobi.png', transparent=True, dpi=72)

percentile_chart('Richarlison') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\FW\Richarlison.png', transparent=True, dpi=72)

percentile_chart('James Rodríguez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\FW\James Rodríguez.png', transparent=True, dpi=72)

In [ ]:
#Fulham

percentile_chart('Ivan Cavaleiro') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Ivan Cavaleiro.png', transparent=True, dpi=72)

percentile_chart('Aboubakar Kamara') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Aboubakar Kamara.png', transparent=True, dpi=72)

percentile_chart('Neeskens Kebano') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Neeskens Kebano.png', transparent=True, dpi=72)

percentile_chart('Ademola Lookman') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Ademola Lookman.png', transparent=True, dpi=72)

percentile_chart('Josh Maja') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Josh Maja.png', transparent=True, dpi=72)

percentile_chart('Aleksandar Mitrović') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Aleksandar Mitrović.png', transparent=True, dpi=72)


In [ ]:
#Leeds United

percentile_chart('Patrick Bamford') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\FW\Patrick Bamford.png', transparent=True, dpi=72)

In [ ]:
#Leicester City

percentile_chart('Harvey Barnes') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\FW\Harvey Barnes.png', transparent=True, dpi=72)

percentile_chart('Kelechi Iheanacho') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\FW\Kelechi Iheanacho.png', transparent=True, dpi=72)

percentile_chart('Jamie Vardy') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\FW\Jamie Vardy.png', transparent=True, dpi=72)

In [ ]:
#Liverpool

percentile_chart('Roberto Firmino') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Roberto Firmino.png', transparent=True, dpi=72)

percentile_chart('Diogo Jota') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Diogo Jota.png', transparent=True, dpi=72)

percentile_chart('Sadio Mané') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Sadio Mané.png', transparent=True, dpi=72)

percentile_chart('Divock Origi') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Divock Origi.png', transparent=True, dpi=72)

#percentile_chart('Alex Oxlade-Chamberlain') 
#plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Alex Oxlade-Chamberlain.png', transparent=True, dpi=72)

percentile_chart('Mohamed Salah') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\FW\Mohamed Salah.png', transparent=True, dpi=72)

In [ ]:
#Manchester City

percentile_chart('Sergio Agüero') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Sergio Agüero.png', transparent=True, dpi=72)

percentile_chart('Phil Foden') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Phil Foden.png', transparent=True, dpi=72)

percentile_chart('Gabriel Jesus') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Gabriel Jesus.png', transparent=True, dpi=72)

percentile_chart('Riyad Mahrez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Riyad Mahrez.png', transparent=True, dpi=72)

percentile_chart('Raheem Sterling') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Raheem Sterling.png', transparent=True, dpi=72)

percentile_chart('Ferrán Torres') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\FW\Ferrán Torres.png', transparent=True, dpi=72)

In [ ]:
#Manchester United

percentile_chart('Edinson Cavani') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Edinson Cavani.png', transparent=True, dpi=72)

percentile_chart('Mason Greenwood') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Mason Greenwood.png', transparent=True, dpi=72)

percentile_chart('Daniel James') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Daniel James.png', transparent=True, dpi=72)

percentile_chart('Anthony Martial') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Anthony Martial.png', transparent=True, dpi=72)

percentile_chart('Juan Mata') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Juan Mata.png', transparent=True, dpi=72)

percentile_chart('Marcus Rashford') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\FW\Marcus Rashford.png', transparent=True, dpi=72)

In [ ]:
#Newcastle United

percentile_chart('Andy Carroll') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\FW\Andy Carroll.png', transparent=True, dpi=72)

percentile_chart('Dwight Gayle') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\FW\Dwight Gayle.png', transparent=True, dpi=72)

percentile_chart('Joelinton') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\FW\Joelinton.png', transparent=True, dpi=72)

percentile_chart('Allan Saint-Maximin') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\FW\Allan Saint-Maximin.png', transparent=True, dpi=72)

percentile_chart('Callum Wilson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\FW\Callum Wilson.png', transparent=True, dpi=72)

percentile_chart('Ryan Fraser') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Ryan Fraser.png', transparent=True, dpi=72)


In [ ]:
#Sheffield United

percentile_chart('Rhian Brewster') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\Rhian Brewster.png', transparent=True, dpi=72)

percentile_chart('Oliver Burke') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\Oliver Burke.png', transparent=True, dpi=72)

percentile_chart('Oliver McBurnie') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\Oliver McBurnie.png', transparent=True, dpi=72)

percentile_chart('David McGoldrick') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\David McGoldrick.png', transparent=True, dpi=72)

percentile_chart('Lys Mousset') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\Lys Mousset.png', transparent=True, dpi=72)

percentile_chart('Billy Sharp') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\FW\Billy Sharp.png', transparent=True, dpi=72)

In [ ]:
#Southampton

percentile_chart('Che Adams') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\FW\Che Adams.png', transparent=True, dpi=72)

percentile_chart('Danny Ings') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\FW\Danny Ings.png', transparent=True, dpi=72)

percentile_chart('Shane Long') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\FW\Shane Long.png', transparent=True, dpi=72)

percentile_chart('Dan Nlundulu') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\FW\Dan Nlundulu.png', transparent=True, dpi=72)

percentile_chart('Nathan Tella') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\FW\Nathan Tella.png', transparent=True, dpi=72)

In [ ]:
#Tottenham

percentile_chart('Gareth Bale') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Gareth Bale.png', transparent=True, dpi=72)

percentile_chart('Steven Bergwijn') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Steven Bergwijn.png', transparent=True, dpi=72)

percentile_chart('Son Heung-min') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Son Heung-min.png', transparent=True, dpi=72)

percentile_chart('Harry Kane') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Harry Kane.png', transparent=True, dpi=72)

percentile_chart('Érik Lamela') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Érik Lamela.png', transparent=True, dpi=72)

percentile_chart('Lucas Moura') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Lucas Moura.png', transparent=True, dpi=72)

percentile_chart('Carlos Vinícius') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\FW\Carlos Vinícius.png', transparent=True, dpi=72)

In [ ]:
#West Brom

percentile_chart('Charlie Austin') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Charlie Austin.png', transparent=True, dpi=72)

percentile_chart('Mbaye Diagne') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Mbaye Diagne.png', transparent=True, dpi=72)

percentile_chart('Karlan Grant') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Karlan Grant.png', transparent=True, dpi=72)

percentile_chart('Kamil Grosicki') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Kamil Grosicki.png', transparent=True, dpi=72)

percentile_chart('Callum Robinson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Callum Robinson.png', transparent=True, dpi=72)

percentile_chart('Hal Robson-Kanu') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\FW\Hal Robson-Kanu.png', transparent=True, dpi=72)

In [ ]:
#West Ham
percentile_chart('Saïd Benrahma') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\FW\Saïd Benrahma.png', transparent=True, dpi=72)

percentile_chart('Michail Antonio') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\FW\Michail Antonio.png', transparent=True, dpi=72)

percentile_chart('Jarrod Bowen') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\FW\Jarrod Bowen.png', transparent=True, dpi=72)

percentile_chart('Pablo Fornals') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\FW\Pablo Fornals.png', transparent=True, dpi=72)

percentile_chart('Andriy Yarmolenko') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\FW\Andriy Yarmolenko.png', transparent=True, dpi=72)

In [ ]:
#Wolves

percentile_chart('Morgan Gibbs-White') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Morgan Gibbs-White.png', transparent=True, dpi=72)

percentile_chart('Raúl Jiménez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Raúl Jiménez.png', transparent=True, dpi=72)

percentile_chart('Willian José') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Willian José.png', transparent=True, dpi=72)

percentile_chart('Pedro Neto') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Pedro Neto.png', transparent=True, dpi=72)

percentile_chart('Fábio Silva') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Fábio Silva.png', transparent=True, dpi=72)

percentile_chart('Adama Traoré') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Adama Traoré.png', transparent=True, dpi=72)

In [ ]:
def directory(squad_name):

    players = passing_percentiles['squad'] == squad_name
    return passing_percentiles[players] 

In [ ]:
directory('Arsenal')